In [1]:
import json
from tqdm import tqdm
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import torch
from sklearn.model_selection import train_test_split

import random

import matplotlib.pyplot as plt
%matplotlib inline
#import plotly.express as px
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [2]:
import psycopg2
con = psycopg2.connect(
database="db_dsa",
user="udsa",
password="P@ssword2021",
host="92.242.58.173",
port="1984"
)

print("Database opened successfully")

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Database opened successfully


In [3]:
cur = con.cursor()
cur.execute("SELECT * from projects")
rows = cur.fetchall()
column_names=["ID","NUMBER","VACANCIES","NAME","TYPE","TYPEDESC","TYPEID","STATUSID", "STATUSDESC","DIRECTIONHEAD","HEAD","DATACREATED","FACULTYID"]
df = pd.DataFrame(rows, columns=column_names)
df.head()

,ID,NUMBER,VACANCIES,NAME,TYPE,TYPEDESC,TYPEID,STATUSID,STATUSDESC,DIRECTIONHEAD,HEAD,DATACREATED,FACULTYID
0,205,NaN,2,Численное и аналитическое исследование эффекта...,nir,НИР,1,1,Готов к работе,Данилов Владимир Григорьевич,Данилов Владимир Григорьевич,08.05.2020 21:25:02,None
1,208,NaN,3,"Исследование механизмов образования, способов ...",nir,НИР,1,1,Готов к работе,,Грачев Николай,11.05.2020 14:07:40,None
2,215,NaN,1,Разработка моста переменного тока,soft-hard,Прогр-аппарат.,3,1,Готов к работе,,Балакин Станислав,13.05.2020 16:41:39,None
3,216,NaN,3,Разработка нательных антенн и излучателей для ...,soft-hard,Прогр-аппарат.,3,1,Готов к работе,Елизаров Андрей Альбертович,Скуридин Андрей,13.05.2020 17:00:19,None
4,218,NaN,0,Разработка глоссария физических терминов для м...,soft,Прогр.,2,1,Готов к работе,,Гузенкова Александра Сергеевна,13.05.2020 19:38:43,None


In [4]:
papers = pd.DataFrame({
    'title': df['NAME'].tolist(),
    'categories': df['TYPE'].tolist()
})

papers['title'] = papers['title'].apply(lambda x: x.replace("\n",""))
papers['title'] = papers['title'].apply(lambda x: x.strip())
papers['text'] = papers['title']

papers['categories'] = papers['categories'].apply(lambda x: tuple(x.split()))

shortlisted_categories = papers['categories'].value_counts().reset_index(name="count").query("count > 0")["index"].tolist()
papers = papers[papers["categories"].isin(shortlisted_categories)].reset_index(drop=True)

papers = papers.sample(frac=1).reset_index(drop=True)

papers = papers.groupby('categories').head(250).reset_index(drop=True)

multi_label_encoder = MultiLabelBinarizer()
multi_label_encoder.fit(papers['categories'])
papers['categories_encoded'] = papers['categories'].apply(lambda x: multi_label_encoder.transform([x])[0])

papers = papers[["text", "categories", "categories_encoded"]]
papers.head()

,text,categories,categories_encoded
0,Приложение поиска и анализа данных в социальны...,"(soft,)","[0, 1, 0, 0]"
1,Видеосервисы МИЭМ,"(soft,)","[0, 1, 0, 0]"
2,Исследование интереса студентов к использовани...,"(nir,)","[1, 0, 0, 0]"
3,"Разработка системы стабилизации установки ""пер...","(soft-hard,)","[0, 0, 1, 0]"
4,Автоматизированное тестирование мобильного при...,"(soft,)","[0, 1, 0, 0]"


In [5]:
papers.shape

(323, 3)

In [6]:
train, test = train_test_split(papers, test_size=0.3, random_state=42)

In [7]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 37.8 MB/s 
     |████████████████████████████████| 4.2 MB 45.1 MB/s 
     |████████████████████████████████| 9.1 MB 52.0 MB/s 
     |████████████████████████████████| 6.6 MB 7.8 MB/s 
     |████████████████████████████████| 346 kB 72.1 MB/s 
     |████████████████████████████████| 1.8 MB 49.1 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 56.5 MB/s 
     |████████████████████████████████| 596 kB 56.8 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 145 kB 50.8 MB/s 
     |████████████████████████████████| 181 kB 16.7 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 140 kB 56.1 MB/s 
     |████████████████████████████████| 1.1 MB 59.5 MB/s 
     |████████████████████████████████

In [8]:
%%time

from simpletransformers.classification import MultiLabelClassificationModel

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False, # Model occupies 1.4GB size per epoch (Total Disk Space Available: 4.9GB)
    "save_eval_checkpoints": False,
    "max_seq_length": 512,
    "train_batch_size": 16,
    "num_train_epochs": 4,
}

# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel('roberta', 
                                      'roberta-base', 
                                      num_labels=len(shortlisted_categories), 
                                      args=model_args)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'c

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

CPU times: user 16.4 s, sys: 2.71 s, total: 19.1 s
Wall time: 37.8 s


In [9]:
model.train_model(train[['text', 'categories_encoded']])

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/226 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_utils.py:263: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  labels = torch.tensor(labels, dtype=torch.long)
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/15 [00:00<?, ?it/s]

(60, 0.4612740745147069)

In [10]:
result, model_outputs, wrong_predictions = model.eval_model(test[['text', 'categories_encoded']])
print(result)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/97 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

{'LRAP': 0.7474226804123713, 'eval_loss': 0.4360346312706287}


In [12]:
for _ in range(5):

    random_idx = random.randint(0, len(test)-1)
    text = test.iloc[random_idx]['text']
    true_categories = test.iloc[random_idx]['categories']

    predicted_categories_encoded, raw_outputs = model.predict([text])
    predicted_categories_encoded = np.array(predicted_categories_encoded)
    predicted_categories_encoded[0][np.argmax(raw_outputs[0])] = 1
    predicted_categories = multi_label_encoder.inverse_transform(predicted_categories_encoded)[0]

    print(f'True Categories:'.ljust(21,' '), f'{true_categories} - {true_categories[0]}\n')
    print(f'Predicted Categories: {predicted_categories} - {predicted_categories[0]}\n')
    print(f'Abstract: {text}\n\n')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

True Categories:      ('soft',) - soft

Predicted Categories: ('soft',) - soft

Abstract: Электронный атлас проводящих путей головного мозга человека




  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

True Categories:      ('soft',) - soft

Predicted Categories: ('soft',) - soft

Abstract: Прогнозирование динамики финансовых инструментов на основе анализа новостей




  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

True Categories:      ('soft',) - soft

Predicted Categories: ('soft',) - soft

Abstract: Мобильное приложение МИЭМ


